In [0]:
from pyspark.sql.functions import *

In [0]:
df_bronze=spark.table("nyc_taxi.nyc_schema.bronze")

In [0]:
df_bronze.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c+'_null_count') for c in df_bronze.columns]).display()

In [0]:
# dropping nulls
df_silver=df_bronze.drop('ehail_fee')
essential_cols=[
    "lpep_pickup_datetime",
    "lpep_dropoff_datetime",
    "trip_distance",
    "fare_amount",
    "total_amount"
]
df_silver=df_silver.dropna(subset=essential_cols)

In [0]:
print(df_silver.count(),df_bronze.count())

In [0]:
# filter invalid rows
df_silver=df_silver.filter(col("trip_distance")>0)\
    .filter(col('fare_amount')>0)\
    .filter(col('total_amount')>0)\
    .filter(col('lpep_pickup_datetime')< col('lpep_dropoff_datetime'))
df_silver.count()

In [0]:
df_silver.display(5)

In [0]:
# Type Casting and Enrichment
df_silver = df_silver.withColumn("pickup_date", to_date("lpep_pickup_datetime")) \
    .withColumn("pickup_hour", hour("lpep_pickup_datetime")) \
    .withColumn("trip_duration_minutes",
        (unix_timestamp("lpep_dropoff_datetime") - unix_timestamp("lpep_pickup_datetime")) / 60
    ) \
    .withColumn("fare_per_mile", round(col("fare_amount") / col("trip_distance"), 2)) \
    .withColumn("tip_percent", round(col("tip_amount") / col("fare_amount"), 2))

df_silver.select("pickup_date","pickup_hour","trip_duration_minutes","fare_per_mile","tip_percent").take(5)

In [0]:
df_silver.show(3)

In [0]:
#adding time based flags
df_silver=df_silver.withColumn("day_of_week",dayofweek('lpep_pickup_datetime'))\
    .withColumn("is_weekend",when(col("day_of_week").isin(1,7),True).otherwise(False))

In [0]:
# adding metadata :processing date
df_silver=df_silver.withColumn("processing_date",current_date())

In [0]:
for field in df_silver.schema.fields:
    print(f"{field.name}: {field.dataType.simpleString()} (nullable={field.nullable})")

In [0]:
# column reordering
final_col_order = [
    "VendorID", "lpep_pickup_datetime", "lpep_dropoff_datetime", "pickup_date", "pickup_hour","day_of_week", "is_weekend", "store_and_fwd_flag", "RatecodeID", "PULocationID", "DOLocationID","passenger_count", "trip_distance", "trip_duration_minutes", "fare_amount", "tip_amount","tip_percent", "tolls_amount", "extra", "mta_tax", "improvement_surcharge",
    "congestion_surcharge","total_amount", "fare_per_mile", "payment_type", "trip_type", "ingestion_date", "source_file", "processing_date"
]

df_silver = df_silver.select(final_col_order)

In [0]:
# drop duplicates
df_silver=df_silver.dropDuplicates()


In [0]:
df_silver.write.format('delta').mode('overwrite').saveAsTable('nyc_taxi.nyc_schema.silver')

In [0]:
%sql
select * from nyc_taxi.nyc_schema.silver limit 1

In [0]:
# reading zone lookup dataset
df_zone_lookup=spark.read.format('csv').option("header", "true").option("inferSchema", "true").load('/Volumes/nyc_taxi/nyc_schema/raw_zone/taxi_zone_lookup.csv')
df_zone_lookup.show(5)

In [0]:
df_zone_lookup.dtypes

In [0]:
df_zone_lookup.select([sum(when(col(c).isNull(),1).otherwise(0)).alias(c+'_null_count') for c in df_zone_lookup.columns]).show()

In [0]:
for c in df_zone_lookup.columns:
    print(f"distinct values in column {c} are {df_zone_lookup.select(c).distinct().orderBy(c).show()}")

In [0]:
df_zone_lookup.columns

In [0]:
df_zone_lookup.count()

In [0]:
df_zone_lookup = df_zone_lookup \
    .withColumnRenamed("LocationID", "location_id") \
    .withColumnRenamed("Borough", "borough") \
    .withColumnRenamed("Zone", "zone") \
    .withColumnRenamed("service_zone", "service_zone")  

In [0]:
# checking for duplicates
df_zone_lookup.groupBy('location_id').count().filter(col('count')>1).show()

In [0]:
df_zone_lookup.write.format('delta').mode('overwrite').saveAsTable('nyc_taxi.nyc_schema.zone_lookup')

In [0]:
%sql
select * from nyc_taxi.nyc_schema.zone_lookup limit 3